# Project Assignment 1: Mustaeen Ahmed

## Goal: Diffuse and mirror reflection due to environmentlight.

## Imports

In [1]:
import pygame
import moderngl
import numpy
import glm
from loadModelUsingAssimp_V2 import create3DAssimpObject

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Intialize

#### Enable anti-aliasing in the GPU pipeline
 - At the time of context creation, request a multisample buffer.: pygame.display.gl_set_attribute(GL_MULTISAMPLEBUFFERS, 1).
 - Specify the number of samples per pixel for multisampling: pygame.display.gl_set_attribute(GL_MULTISAMPLESAMPLES, 16). Higher the number better the antialiasing. Check the max number of samples supported in your driver. (ctx.max_samples)

In [2]:
width = 840
height = 480

pygame.init() # Initlizes its different modules. Display module is one of them.
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLEBUFFERS, 1)
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLESAMPLES, 16)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Project Assignment 01: Mustaeen Ahmed")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

'4.6.0 NVIDIA 555.99'

## Reflecting Object: 
Uses a Dummy Fragment shader code.  
It must be modified for computing lighting due to environment light. See the assignment details.

In [3]:
#
# Vertex shader(s)
#
vertex_shader_code = '''
#version 460 core
layout (location=0) in vec3 position;
layout (location=1) in vec3 normal;
layout (location=2) in vec2 uv;


uniform mat4 model, view, perspective;
uniform mat3 normalMatrix;

out vec2 f_uv;
out vec3 f_normal; // Normal vector in World Coordinates
out vec3 f_position; // postion in world coordinates
void main() {
    f_uv = uv;
    vec4 P = model*vec4(position, 1);
    f_position = P.xyz;
    gl_Position = perspective*view*P;
    mat3 normalMatrix = mat3(transpose(inverse(model)));// inverse transpose of model transformation
    f_normal = normalize(normalMatrix*normal);
}
'''

#
# Fragment shader(s)
#
fragment_shader_code = '''
#version 460 core

in vec3 f_normal;
in vec3 f_position;
in vec2 f_uv;

uniform sampler2D map;
uniform samplerCube cubeMapEnv;
uniform vec3 eye_position;
uniform bool metal;

uniform vec3 skyColor;
uniform vec3 groundColor;
uniform vec3 upDir;

out vec4 out_color;

void main() {
    vec3 N = normalize(f_normal);
    vec3 V = normalize(eye_position - f_position);

    if (metal) {
        vec3 R = reflect(-V, N);
        vec3 envColor = texture(cubeMapEnv, normalize(R)).rgb;

        vec3 specTint = texture(map, f_uv).rgb;
        vec3 color = specTint * envColor;

        out_color = vec4(color, 1.0);
    } else {
        float w = 0.5 * (1.0 + dot(N, normalize(upDir)));
        w = clamp(w, 0.0, 1.0);
        vec3 incident = mix(groundColor, skyColor, w);

        vec3 base = vec3(1.0);
        vec3 color = incident * base;
        out_color = vec4(color, 1.0);
    }
}   
'''
#
# Programs
#

model_program = gl.program(
    vertex_shader= vertex_shader_code,
    fragment_shader= fragment_shader_code
)
format = "3f 3f 2f"
variables = ["position","normal", "uv"]

### Use Assimp parser and and read a 3D model.
modelFile = "the_utah_teapot/scene.gltf" 
modelObj = create3DAssimpObject(modelFile, verbose=False, textureFlag = True, normalFlag = True)

model_renderables = modelObj.getRenderables(gl, model_program, format, variables)
scene = modelObj.scene

#### Recursive Render for scenegraph

def recursive_render(node, M):
    nodeTransform = glm.transpose(glm.mat4(node.transformation));
    currentTransform = M * nodeTransform
    if node.num_meshes > 0:
        for index in node.mesh_indices:
            model_renderables[index]._program["model"].write(currentTransform)         
            model_renderables[index].render()
            
    for node in node.children:
        recursive_render(node, currentTransform)

def render():
    recursive_render(scene.root_node, M=glm.mat4(1))
    
_imageFile = "gold.jpg"
_texture_img = pygame.image.load(_imageFile) 
_texture_data = pygame.image.tobytes(_texture_img,"RGB", True) 
_texture = gl.texture(_texture_img.get_size(), data = _texture_data, components=3)
_texture.build_mipmaps()
gold_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x=True, repeat_y=True)
bound = modelObj.bound

Accumulates Normals in the geomDataList
Accumulates Textures in the geomDataList


In [4]:
def cubemap_sampler(base_dir="Footballfield"):
    faces = ["posx","negx","posy","negy","posz","negz"]
    blob = bytearray()
    last_size = None
    for f in faces:
        path = f"{base_dir}/{f}.jpg"
        img = pygame.image.load(path)
        if last_size is None:
            last_size = img.get_size()
        else:
            assert img.get_size() == last_size, "All cubemap face images must have the same size"
        data = pygame.image.tobytes(img, "RGB", False)
        blob.extend(data)
    tex = gl.texture_cube(last_size, components=3, data=blob)
    tex.build_mipmaps()
    return gl.sampler(texture=tex, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x=True, repeat_y=True)

cube_sampler = cubemap_sampler()

### Skybox rendering template: 
Uses a dummy vertex shader and a dummy fragment shader.  
It must be modified to carry out skybox rendering.

In [5]:
_positions = numpy.array([
    [-1, 1],
    [ 1, 1],
    [ 1,-1],
    [-1,-1]
]).astype("float32")

_geom = _positions.flatten()

_index = numpy.array([
    0, 1, 2,
    2, 3, 0
]).astype("int32")

#
# Vertex shader(s)
#
_vertex_shader_code = '''
#version 460 core
in vec2 position;

uniform mat4 inversePV;
uniform vec3 eye_position;

out vec3 Vdir;

void main() {
    vec4 clipPos = vec4(position, 1.0, 1.0);
    gl_Position = clipPos;

    vec4 P = inversePV * clipPos;
    vec3 world = P.xyz / P.w;
    Vdir = normalize(world - eye_position);
 }
'''
#
# Fragment shader(s)
#
_fragment_shader_code = '''
#version 460 core
in vec3 Vdir;
uniform samplerCube cubeMapEnv;

out vec4 out_color;

void main() {
    vec3 c = texture(cubeMapEnv, normalize(Vdir)).rgb;
    out_color = vec4(c, 1.0);
}

'''
skybox_program = gl.program(
    vertex_shader= _vertex_shader_code,
    fragment_shader= _fragment_shader_code
)

skybox_renderable = gl.vertex_array(skybox_program,
                [(gl.buffer(_geom), "2f", "position")],
                index_buffer=gl.buffer(_index),index_element_size=4
            )

### Define Camera Parameters

In [6]:
displacement_vector = 2*bound.radius*glm.rotate(glm.vec3(0,1,0), glm.radians(85), glm.vec3(1,0,0)) #glm.vec3(0,0,1) 

light_displacement_vector = 2*bound.radius*glm.rotate(glm.vec3(0,1,0), glm.radians(45), glm.vec3(1,0,0)) 
    
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

### View volume parameters
fov_radian = glm.radians(45) # In radian
aspect = width/height
near = bound.radius
far = 3*bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

### Render loop

In [ ]:
running = True
clock = pygame.time.Clock()
alpha = 0
lightAngle = 0

pause = True   # Keyboard key "p" toggles pause/orbit
metal = False  # Keyboard key "m" toggles metal/diffuse
skybox = False # Keyboard key "s" toggles skybox/noSkybox

gl.depth_func = '<=' 
gl.enable(gl.DEPTH_TEST)

model_program["skyColor"].write(glm.vec3(0.718, 0.741, 0.753))
model_program["groundColor"].write(glm.vec3(0.322, 0.4, 0.11))
model_program["upDir"].write(glm.vec3(0.0, 1.0, 0.0))

while running:   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type ==  pygame.KEYDOWN):
            if  event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif event.key == pygame.K_s:
                skybox = not skybox
            elif event.key == pygame.K_m:
                metal = not metal
            elif event.key == pygame.K_LEFT:
                lightAngle -= 5
            elif event.key == pygame.K_RIGHT:
                lightAngle += 5
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            aspect = width / height if height > 0 else 1.0
            perspectiveMatrix = glm.perspective(fov_radian, width/height, near, far)
            gl.viewport = (0, 0, width, height)

    # create the aspect ratio correction matrix
    new_displacement_vector = glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0,1,0))

    new_light_displacement_vector = glm.rotate(light_displacement_vector, glm.radians(lightAngle), glm.vec3(0,1,0))
    
    eye_point = target_point + new_displacement_vector

    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.5,0.5,0.0, depth=1.0)
    gl.depth_func = '<=' 

    if skybox:
        pv = perspectiveMatrix * viewMatrix
        inversepv = glm.inverse(pv)

        skybox_program["inversePV"].write(inversepv)
        skybox_program["eye_position"].write(eye_point)
        cube_sampler.use(1)
        skybox_program["cubeMapEnv"].value = 1

        gl.depth_mask = False
        gl.disable(moderngl.DEPTH_TEST)
        skybox_renderable.render()

        gl.enable(moderngl.DEPTH_TEST)
        gl.depth_mask = True
        gl.depth_func = '<=' 

    # Render Relfector
    program = model_program
    program["view"].write(viewMatrix)
    program["perspective"].write(perspectiveMatrix)   
    program["eye_position"].write(eye_point)
    program["metal"].value = 1 if metal else 0

    gold_sampler.use(0)
    cube_sampler.use(1)
    program["map"] = 0
    model_program["cubeMapEnv"].value = 1
    render()
    
    pygame.display.flip()
    clock.tick(60)  # limits FPS to 10
    if not pause:
        alpha +=  1
        if alpha > 360:
            alpha = 0
    #running = False
pygame.display.quit()